# 5 数据类构建器

Python提供了几种构建简单类的方式，这些类只是字段的容器，几乎没有额外功能。这种模式叫做数据类（data class）。

例如
collections.namedtuple
typing.NamedTuple
@dataclasses.dataclass

## 5.2 数据类构建器概述

In [1]:
# 例如构建一个只存放经纬度的类，一般写法为
class Coordinate:
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long

moscow = Coordinate(55.7558, 37.6176)
location = Coordinate(55.7558, 37.6176)

# 无意义的__repr__方法
print(moscow)
# 不能直接比较，因为实例的ID不同，只能逐一比较属性
print(moscow == location)
print((moscow.lat, moscow.long) == (location.lat, location.long))

False
True


In [7]:
# 使用collections.namedtuple构建
from collections import namedtuple
Coordinate = namedtuple('Coordinate', 'lat long')
# 元类方法创建类
print(issubclass(Coordinate, tuple))
moscow = Coordinate(55.7558, 37.6176)
location = Coordinate(55.7558, 37.6176)
# 有意义的__repr__方法
print(moscow)
# 可以直接比较
print(moscow == location)

True
Coordinate(lat=55.7558, long=37.6176)
True


In [6]:
# 使用typing.NamedTuple构建,还可以给属性加类型注解
from typing import NamedTuple
Coordinate = NamedTuple('Coordinate', [('lat', float), ('long', float)])

# 元类方法创建类
print(issubclass(Coordinate, tuple))
moscow = Coordinate(55.7558, 37.6176)
location = Coordinate(55.7558, 37.6176)
# 有意义的__repr__方法
print(moscow)
# 可以直接比较
print(moscow == location)

# 也可以使用继承的写法
class Coordinate(NamedTuple):
    lat: float
    long: float

    def __str__(self) -> str:
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.long >= 0 else 'W'
        return f'{abs(self.lat):.1f}{ns}, {abs(self.long):.1f}{we}'

# 元类方法创建类
print(issubclass(Coordinate, tuple))
moscow = Coordinate(55.7558, 37.6176)
location = Coordinate(55.7558, 37.6176)
# 有意义的__repr__方法
print(moscow)
# 可以直接比较
print(moscow == location)

True
Coordinate(lat=55.7558, long=37.6176)
True
True
55.8N, 37.6E
True


In [10]:
# 使用dataclasses.dataclass构建
from dataclasses import dataclass

# @dataclass装饰器不依赖继承和元类
@dataclass
class Coordinate:
    lat: float
    long: float

    def __str__(self) -> str:
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.long >= 0 else 'W'
        return f'{abs(self.lat):.1f}{ns}, {abs(self.long):.1f}{we}'

print(issubclass(Coordinate, object))
moscow = Coordinate(55.7558, 37.6176)
location = Coordinate(55.7558, 37.6176)
# 有意义的__repr__方法
print(moscow)
# 可以直接比较
print(moscow == location)

True
55.8N, 37.6E
True


### 三种方式的主要功能区别
1. 可变实例
    collections.namedtuple和typing.NamedTuple构建的类是tuple的子类，因此实例不可变。
    @dataclasses.dataclass默认构建可变的类。不过如果指定参数frozen=True，也可以构建不可变的类。
    @dataclasses.dataclass(frozen=True)

2. class语句句法
    只有typing.NamedTuple和@dataclasses.dataclass使用class语句句法。

3. 构造字典
    两种具名元组都可以通过._asdict()方法构造字典。
    dataclass实例可以通过dataclasses.asdict()函数构造字典。

4. 获取字段名称和默认值
    3种方式都支持获取字段名称和可能配置的默认值。
    具名元组类中，这些数据在._fields和._field_defaults属性中。
    dataclass类中，可以用过fields()函数获取。

5. 获取字段类型
    typing.NamedTuple和@dataclasses.dataclass定义的类都有一个__annotations__属性，其中包含字段名和类型的映射。
    不要直接访问__annotations__属性，而是使用typing.get_type_hints()函数。

6. 更改之后创建新实例
    对于具名元组实例x，x._replace(**kwargs)根据指定的关键字参数替换某些属性的值，然后返回新的实例。
    dataclasses.replace(x, **kwargs)与dataclass装饰的类具有相同的作用。

7. 运行时定义新类
    运行时动态构建数据类，可以使用默认的函数调用句法，collections.namedtuple和typing.NamedTuple都支持这种方式。
    dataclasses.make_dataclass()函数也支持这种方式。

## 5.3 典型的具名元组（collections.namedtuple）


In [19]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
print(tokyo)
print(tokyo.coordinates)
print(tokyo[1])
#查询所有的属性名
print(City._fields)
#使用_make()通过接受一个可迭代对象来生成一个新的实例
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))
# 等同于delhi = City('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
delhi = City._make(delhi_data)
# 使用_asdict()返回dict
print(delhi._asdict())

import json
# 转json
print(json.dumps(delhi._asdict()))

# 为字段指定默认值 3.7开始可以用defaults参数，值为一个产生N项的可迭代对象，为从右数第N个字段指定默认值
Coordinate = namedtuple('Coordinate', 'lat long', defaults=[1,2])
print(Coordinate())
Coordinate = namedtuple('Coordinate', 'lat long', defaults=[1])
print(Coordinate(2))

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))
(35.689722, 139.691667)
JP
('name', 'country', 'population', 'coordinates')
{'name': 'Delhi NCR', 'country': 'IN', 'population': 21.935, 'coordinates': LatLong(lat=28.613889, long=77.208889)}
{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}
Coordinate(lat=1, long=2)
Coordinate(lat=2, long=1)


## 5.4 带类型的具名元组（typing.NamedTuple）

In [ ]:
from typing import NamedTuple

# 每个字段都有类型注解
class Coordinate(NamedTuple):
    lat: float
    long: float 
    # 为字段指定默认值
    reference: str = 'WGS84'

使用typing.NamedTuple构建的类与collections.namedtuple生成的类的方法基本相同，区别就是多了类属性__annotations__。

## 5.5 类型提示入门

Python编译器和解释器并不强制要求提供类型信息。

### 5.5.1 运行时没有作用

Python类型提示可以看作“供IDE和类型检查工具验证类型的文档”，对运行时没有任何作用。

In [20]:
import typing
class Coordinate(typing.NamedTuple):
    lat: float
    long: float 
    # 为字段指定默认值
    reference: str = 'WGS84'

# 运行时并不检查类型
trash = Coordinate("lal", "la")
print(trash)

Coordinate(lat='lal', long='la', reference='WGS84')


### 5.5.2 变量注解句法

基本句法 
var_name: var_type

定义数据类时常用三种类型
- 一个具体类，str FrenchDeck
- 一个参数化的容器类型，list\[str\] tuple\[int, int\]
- typing.Optional 用于可选参数 Optional\[int\] 可以是int或None

### 5.5.3 变量注解的意义


In [27]:
# 普通类
class DemoPlainClass:
    a: int
    b: float = 1.1
    c = 'spam'

# a只作为注解存在，不是类属性，b和c是类属性
print(DemoPlainClass.__annotations__)
try:
# 作者还嘲讽了JS的undefined,是最大的败笔之一
    print(DemoPlainClass.a)
except AttributeError as e:
    print(e)
print(DemoPlainClass.b)
print(DemoPlainClass.c)

# typing.NamedTuple类
import typing
class DemoNTClass(typing.NamedTuple):
    # a是注解也是类属性
    a: int
    b: float = 1.1
    c = 'spam'

print(DemoNTClass.__annotations__)
# a和b为描述符，描述符可以理解为特性（property）的读值（getter）方法
print(DemoNTClass.a)
print(DemoNTClass.b)
print(DemoNTClass.c)

print(DemoNTClass.__doc__)


# dataclass类
from dataclasses import dataclass
@dataclass
class DemoDCClass:
    # a是注解也是受描述符控制的实例属性
    a: int
    b: float = 1.1
    c = 'spam'

print(DemoDCClass.__annotations__)
print(DemoDCClass.__doc__)
try:
    # 只是实例属性，不是类属性
    print(DemoDCClass.a)
except AttributeError as e:
    print(e)
print(DemoDCClass.b)
print(DemoDCClass.c)

dc=DemoDCClass(1)
print(dc.a)
# 可变的
dc.a = 2
print(dc.a)
# 还能新增属性
dc.x = 3
print(dc.x)

{'a': <class 'int'>, 'b': <class 'float'>}
type object 'DemoPlainClass' has no attribute 'a'
1.1
spam
{'a': <class 'int'>, 'b': <class 'float'>}
_tuplegetter(0, 'Alias for field number 0')
_tuplegetter(1, 'Alias for field number 1')
spam
DemoNTClass(a, b)
{'a': <class 'int'>, 'b': <class 'float'>}
DemoDCClass(a: int, b: float = 1.1)
type object 'DemoDCClass' has no attribute 'a'
1.1
spam
1
2
3


## 5.6 @dataclass详解

@dataclass装饰器接受多个参数
@dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
- init：是否生成__init__方法，默认为True
- repr：是否生成__repr__方法，默认为True
- eq：是否生成__eq__方法，默认为True
- order：是否生成__lt__、__le__、__gt__、__ge__方法，默认为False
- unsafe_hash：是否强制生成__hash__方法，默认为False，有很多问题需要注意
- frozen：是否生成不可变类，默认为False

如果eq=True，frozen=True，那么@dataclass将生成一个__hash__方法，确保实例可哈希。
对于frozen=False，@dataclass会将__hash__方法设置为None。

### 5.6.1 字段选项

@dataclass装饰器拒绝以下形式定义类


In [29]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    #错误的默认值写法
    # guests: list = []
    # 正确的默认值写法
    guests: list = field(default_factory=list)



default_factory参数可以是一个函数、一个类，或者其他可调用对象，在每次创建数据类的实例时调用（不带参数），构建默认值。
default_factory是field函数最常使用的参数，但是field函数还有其他参数。
- default：字段的默认值 
- default_factory：不接受参数的函数
- init：把字段作为参数传给__init__，默认为True
- repr：是否在repr中显示字段，默认为True
- compare：是否在比较中使用字段，默认为True
- hash：是否在哈希中使用字段，默认为None
- metadata：用户定义的数据映射，@dataclass忽略该参数

### 5.6.2 初始化后处理
如果初始化@dataclass类的实例时，需要进行一些额外的处理，可以定义__post_init__方法, @dataclass会在__init__方法最后调用__post_init__方法。常用于执行验证或者根据其他字段计算一个字段的值。

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    #错误的默认值写法
    # guests: list = []
    # 正确的默认值写法
    guests: list = field(default_factory=list)

@dataclass
class HackerClubMember(ClubMember):
    all_handles = set()
    handle: str = ''

    def __post_init__(self):
        cls = self.__class__
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:
            msg = f'handle {self.handle!r} already in use'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)

### 5.6.3 带类型的类属性

    类型检查工具可能对上面例子的类属性定义报错。
    可以使用 all_handles: ClassVar[Set[str]] = set() 定义类属性。
    类型为ClassVar时，@dataclass不会生成实例属性。

### 5.6.4 初始化不作为字段的变量

    仅做初始化的变量（init-only variable）意思为不作为实例字段的参数。可以使用伪类型InitVar定义。

In [32]:
from dataclasses import InitVar, dataclass


@dataclass
class C:
    i: int
    j: int = None
    database: InitVar[str] = None

    def __post_init__(self, database):
        if self.j is None and database is not None:
            self.j = database

c = C(10, database='db')
print(c)

C(i=10, j='db')


### 5.6.5 @dataclass示例：都柏林核心模式

都柏林核心模式是小组术语，可用于描述数字资源（视频、图像、网页等），也可用于描述物理资源，例如图书、CD和艺术品等对象。

In [35]:
from dataclasses import dataclass, field, fields
from typing import Optional
from enum import Enum, auto
from datetime import date


class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()

@dataclass
class Resource:
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)

    def __repr__(self) -> str:
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']
        for f in fields(cls):
            value = getattr(self, f.name)
            res.append(f'{indent}{f.name} = {value!r},')

        res.append(')')
        return '\n'.join(res)

description = 'The Zen of Python, by Tim Peters'
book = Resource('978-0-13-213080-6', 'The Zen of Python', ['Tim Peters'], date(2004, 3, 25), ResourceType.BOOK, description, 'English', ['Python (Computer program language)'])

print(book)

Resource(
    identifier = '978-0-13-213080-6',
    title = 'The Zen of Python',
    creators = ['Tim Peters'],
    date = datetime.date(2004, 3, 25),
    type = <ResourceType.BOOK: 1>,
    description = 'The Zen of Python, by Tim Peters',
    language = 'English',
    subjects = ['Python (Computer program language)'],
)
Resource(
    identifier = '978-0-13-213080-6',
    title = 'The Zen of Python',
    creators = ['Tim Peters'],
    date = datetime.date(2004, 3, 25),
    type = <ResourceType.BOOK: 1>,
    description = 'The Zen of Python, by Tim Peters',
    language = 'English',
    subjects = ['Python (Computer program language)'],
)


## 5.7 数据类导致代码异味

当使用了数据类时可能代表设计存在问题。

所谓数据类是指，他们拥有一些字段，以及用于访问这些字段的函数，除此之外一无长物。这样的类只是一种不会说话的数据容器，他们几乎一定被其他类过分繁琐的操控着。

遇到数据类，请问自己一个问题：这个类需要什么行为？然后开始重构，加入需要的行为，将一个空洞的对象抽象为真正的类。

面向对象编程的主要思想就是把行为和数据放在同一个代码单元中。如果一个类使用广泛，但是没有任何行为，那么整个系统中可能遍布处理实例的代码，并且出现在很多方法和函数中。这样的系统很难维护。

### 5.7.1 把数据类用作脚手架

刚开始创建一个项目时，先用数据类创建简单的类，随着时间的推移，再逐渐添加行为。

### 5.7.2 把数据类用作中间表述
数据类可用于构建将要导出为JSON或其他交换格式的记录，也可用于存储刚刚从其他系统导入的数据。
在这种情况下，一般将数据类实例当作不可变对象处理。

## 5.8 模式匹配类实例

类模式通过类型和属性匹配类实例。类模式匹配的对象可以是任何类的实例。

### 5.8.1 简单类模式

match x:
    case float():
        print('x is a float')

### 5.8.2 关键字类模式


In [36]:
import typing

class City(typing.NamedTuple):
    continent: str
    name: str
    country: str

cities = [
    City('Asia', 'Tokyo', 'JP'),
    City('Europe', 'Moscow', 'RU'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('North America', 'San Francisco', 'US'),
    City('Europe', 'London', 'UK'),
    City('Asia', 'Shanghai', 'CN'),
    City('Europe', 'Paris', 'FR'),
]

def match_asian_cities(cities: list[City]) -> list[City]:
    result = []
    for city in cities:
        match city:
            case City(continent='Asia'):
                result.append(city)

    return result

print(match_asian_cities(cities))

### 5.8.3 位置类模式
def match_asian_cities(cities: list[City]) -> list[City]:
    result = []
    for city in cities:
        match city:
            case City('Asia'):
                result.append(city)

    return result

[City(continent='Asia', name='Tokyo', country='JP'), City(continent='Asia', name='Delhi', country='IN'), City(continent='Asia', name='Shanghai', country='CN')]
